In [ ]:
!ls ../input/tf-installing-packageofficial-model-and-text

In [ ]:
!mkdir -p /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/cached_property-1.5.2-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/gast-0.4.0-py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/gin_config-0.4.0-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/grpcio-1.34.1-cp37-cp37m-manylinux2014_x86_64.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/h5py-3.1.0-cp37-cp37m-manylinux1_x86_64.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/portalocker-2.0.0-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/py-cpuinfo-8.0.0.xyz.csv /tmp/pip/cache/py-cpuinfo-8.0.0.tar.gz
!cp ../input/tf-installing-packageofficial-model-and-text/pycocotools-2.0.2.xyz.csv /tmp/pip/cache/pycocotools-2.0.2.tar.gz
!cp ../input/tf-installing-packageofficial-model-and-text/sacrebleu-1.5.1-py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/seqeval-1.2.2-py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tensorboard-2.5.0-py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tensorflow-2.5.0-cp37-cp37m-manylinux2010_x86_64.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tensorflow_estimator-2.5.0-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tensorflow_model_optimization-0.5.0-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tensorflow_text-2.5.0-cp37-cp37m-manylinux1_x86_64.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tf_models_official-2.5.0-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tf_slim-1.1.0-py2.py3-none-any.whl /tmp/pip/cache/

In [ ]:
!mkdir -p /tmp/pip/cache/
!pip install --no-index --find-links /tmp/pip/cache/ tensorflow-text
!pip install --no-index --find-links /tmp/pip/cache/ tf-models-official

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
from tqdm import tqdm
import os
import glob
import datetime
import json
import re
from sklearn.model_selection import train_test_split

In [ ]:
#tfhub_handle_encoder = "/kaggle/input/bertpreprocessing-v3-en-uncased-l12-v4/bert_en_uncased_L-12_H-768_A-12_4"
#tfhub_handle_preprocess = "/kaggle/input/bertpreprocessing-v3-en-uncased-l12-v4/bert_en_uncased_preprocess_3"

In [ ]:
df = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
df

In [ ]:
df = df.drop(['url_legal','license'], axis =1)
df

In [ ]:
def clean_text(text):
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)#remove url
    text = re.sub(r"\S+@\S+", ' ', text) #remove email address
    text = re.sub(r"[^a-zA-Z.!?']", ' ', text) #remove special symbol
    text = re.sub(r" +", ' ', text)
    return text

In [ ]:
df['excerpt'] = df['excerpt'].apply(lambda x: clean_text(x))
df

In [ ]:
regressor = tf.saved_model.load('../input/pre-trained-commonlite/L24(loss_0.2988)')

In [ ]:
test_text = df['excerpt']
test_text

In [ ]:
def get_prediction(text):
    result = tf.constant([text])
    result = regressor(result)
    result = result.numpy()
    result = result[0]
    
    return result

In [ ]:
prediction = test_text.apply(lambda x: get_prediction(x))

In [ ]:
target = []
for i in range(len(prediction)):
    target.append(prediction[i][0])
target

In [ ]:
submission = pd.DataFrame(zip(df['id'],target),
                         columns =['id', 'target'])
submission

In [ ]:
submission.to_csv('submission.csv',index=False)